<a href="https://colab.research.google.com/github/jonathanahc/Proyecto/blob/main/ProyectoProgADAP_255497.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto de Programación para Analítica Descriptiva y Predictiva**
## Nombre del Proyecto: Consumo y Costo del Agua en Nueva York
## Grupo: A
## Autor: Jonathan Adrian Herrera Castro
## Matrícula: 255497
## Fecha: 19/02/2025

# **1. Introducción**

## **- Descripción del problema:** el objetivo de este análisis es examinar el consumo de agua en la ciudad de Nueva York y su costo asociado, con el fin de identificar tendencias y algunos factores que influyen en su variación. Se analizarán patrones de consumo en distintas zonas y sectores de febrero del 2013 hasta 2023.

## **- Objetivo del análisis:** comprender el consumo y costo del agua en Nueva York para identificar patrones y factores que afectan su variabilidad. Este análisis tiene como impacto en el tema de la optimización el uso del agua, la planificación de tarifas y establecer algunas estrategias de sostenibilidad.

## **- Fuente de datos:** los datos utilizados en este análisis provienen de NYC Open Data, una plataforma que ofrece bases de datos de acceso público. La información fue proporcionada por la Autoridad de Vivienda de la Ciudad de Nueva York (NYCHA, por sus siglas en inglés).


# **2. Captura y Carga de Datos**

**En esta fase de carga de datos, se emplearon diversas librerías que facilitaron el proceso de carga del archivo. La primera librería utilizada fue google.colab, la cual establece la conexión a la carpeta Drive donde se encuentra almacenado el archivo.**

In [96]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Después, se importó la librería Pandas.**

In [113]:
import pandas as pd
import re

**Con esta sentencia, se cargó el archivo CSV en un dataframe sin presentar inconvenientes durante el proceso.**

---



In [114]:
df = pd.read_csv('/content/drive/MyDrive/ProyectoProgramacion/Water_Consumption_And_Cost__2013_-_Feb_2023__20250216.csv', engine="python")

**Una vez cargado el archivo, se validó que los datos se mostrarán correctamente.**

In [115]:
df.head()

,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,...,Service End Date,# days,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (HCF),Water&Sewer Charges,Other Charges
0,HOWARD AVENUE,BROOKLYN,HOWARD AVENUE,BLD 02,AMR,NaN,339.0,782,K033900,FEDERAL,...,04/23/2020,31.0,E11310572,N,2945.22,Basic Water and Sewer,Yes,285,2945.22,0.0
1,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,...,01/26/2020,34.0,K13060723,N,196.35,Basic Water and Sewer,Yes,19,196.35,0.0
2,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,...,02/24/2020,29.0,K13060723,N,258.35,Basic Water and Sewer,Yes,25,258.35,0.0
3,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,...,03/23/2020,28.0,K13060723,N,217.02,Basic Water and Sewer,Yes,21,217.02,0.0
4,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,...,04/23/2020,31.0,K13060723,N,103.34,Basic Water and Sewer,Yes,10,103.34,0.0


**Utilizando la función Shape, se realizó un conteo del número de observaciones y variables del dataframe.**

In [116]:
print(f"Observaciones: {df.shape[0]}, Variables: {df.shape[1]}")

Observaciones: 50315, Variables: 25


**Después del conteo, se utilizó la función Info con la cual se obtuvó la información sobre las columnas, incluyendo el nombre, conteo de valores no nulos y el tipo de dato de cada una.**

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50315 entries, 0 to 50314
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Development Name     50255 non-null  object 
 1   Borough              50315 non-null  object 
 2   Account Name         50315 non-null  object 
 3   Location             49487 non-null  object 
 4   Meter AMR            49805 non-null  object 
 5   Meter Scope          12782 non-null  object 
 6   TDS #                50255 non-null  float64
 7   EDP                  50315 non-null  int64  
 8   RC Code              50315 non-null  object 
 9   Funding Source       50239 non-null  object 
 10  AMP #                50193 non-null  object 
 11  Vendor Name          50315 non-null  object 
 12  UMIS BILL ID         50315 non-null  int64  
 13  Revenue Month        50315 non-null  object 
 14  Service Start Date   50308 non-null  object 
 15  Service End Date     50308 non-null 

#**3.Transformación y Preprocesamiento - Data Wrangling**

In [118]:
df.describe()

,TDS #,EDP,UMIS BILL ID,# days,Current Charges,Consumption (HCF),Water&Sewer Charges,Other Charges
count,50255.000000,50315.000000,5.031500e+04,50308.000000,50315.000000,50315.000000,50315.000000,50315.000000
mean,258.871794,463.205764,6.597390e+06,33.182953,2150.221066,224.291066,2140.327436,9.893631
std,101.467018,185.109210,2.671463e+06,53.181840,5623.852543,2020.962056,5619.812518,607.714232
min,1.000000,200.000000,2.576038e+06,-335.000000,-1748.510000,0.000000,0.000000,-67726.830000
25%,209.000000,325.000000,4.132859e+06,29.000000,46.460000,5.000000,46.360000,0.000000
50%,273.000000,396.000000,6.355677e+06,30.000000,197.360000,26.000000,196.350000,0.000000
75%,338.000000,637.000000,8.627548e+06,32.000000,2598.415000,236.000000,2575.520000,0.000000
max,559.000000,999.000000,1.217584e+07,6968.000000,392036.460000,434200.000000,392036.460000,77796.690000


**Columnas**

In [119]:
df.dtypes

,0
Development Name,object
Borough,object
Account Name,object
Location,object
Meter AMR,object
Meter Scope,object
TDS #,float64
EDP,int64
RC Code,object
Funding Source,object


In [120]:
df.columns = df.columns.str.lower().str.replace(r"\W+", "", regex=True)

In [121]:
df.columns

Index(['developmentname', 'borough', 'accountname', 'location', 'meteramr',
       'meterscope', 'tds', 'edp', 'rccode', 'fundingsource', 'amp',
       'vendorname', 'umisbillid', 'revenuemonth', 'servicestartdate',
       'serviceenddate', 'days', 'meternumber', 'estimated', 'currentcharges',
       'rateclass', 'billanalyzed', 'consumptionhcf', 'watersewercharges',
       'othercharges'],
      dtype='object')

In [122]:
memoria = df.memory_usage(deep=True)
memoria_df = pd.DataFrame({'Columna': memoria.index, 'Bytes': memoria.values})
memoria_df

,Columna,Bytes
0,Index,132
1,developmentname,4158857
2,borough,3139336
3,accountname,4163255
4,location,3485202
5,meteramr,3011043
6,meterscope,2035361
7,tds,402520
8,edp,402520
9,rccode,3220160


In [123]:
df['tds'] = df['tds'].fillna(0).astype(int)
df['days'] = df['days'].fillna(0).astype(int)
df['servicestartdate'] = pd.to_datetime(df['servicestartdate'], format='%m/%d/%Y')
df['serviceenddate'] = pd.to_datetime(df['serviceenddate'], format='%m/%d/%Y')
df['revenuemonth'] = pd.to_datetime(df['revenuemonth'], format='%Y-%m')

In [124]:
columnas = df.select_dtypes(include=['object']).columns

for col in columnas:
    df[col] = df[col].astype('category')

In [125]:
memoria = df.memory_usage(deep=True)
memoria_df = pd.DataFrame({'Columna': memoria.index, 'Bytes': memoria.values})
memoria_df

,Columna,Bytes
0,Index,132
1,developmentname,63174
2,borough,51082
3,accountname,63867
4,location,131548
5,meteramr,50615
6,meterscope,56276
7,tds,402520
8,edp,402520
9,rccode,61779


In [126]:
df.dtypes

,0
developmentname,category
borough,category
accountname,category
location,category
meteramr,category
meterscope,category
tds,int64
edp,int64
rccode,category
fundingsource,category


**Valores nulos**

In [127]:
df.isnull().sum()

,0
developmentname,60
borough,0
accountname,0
location,828
meteramr,510
meterscope,37533
tds,0
edp,0
rccode,0
fundingsource,76


# **4. Análisis Descriptivo de Datos**

## - Preguntas a responder con el AD
## - Explicación de Estadísticas Básicas Aplicadas, Visualización de Datos con Gráficos, y Resumen de las respuestas

# **5. Análisis Exploratorio de los Datos**

## - Preguntas a responder
## - Estadísticas Empleadas, visualización y Resumen de las respuestas encontradas.

# **6. Análisis Predictivo**

## - Preprocesamiento para Análisis Predictivo: Problemas, Técnicas y Código

## - Modelado Predictivo
### - Modelo elegido y parámetros
### - Entrenamiento y Evaluación
## - Resultados


# **7. Conclusiones y Trabajos Futuros**

## - Resumen del trabajo realizado, hallazgos y posibles mejoras